In [1]:
import pandas as pd
import numpy as np

# Naive Bayes with Womens Clothing E-commerce Reviews dataset

In [2]:
review = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
review.drop(['Unnamed: 0'], axis = 1, inplace = True)
review.head()

,Clothing ID,Age,Title,Review Text,Rating,Positive Feedback Count,Division Name,Department Name,Category,Recommended IND
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,0,Initmates,Intimate,Intimates,1
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,4,General,Dresses,Dresses,1
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses,0
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,0,General Petite,Bottoms,Pants,1
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,6,General,Tops,Blouses,1


## 1. Preprocessing

In [3]:
review.isnull().sum()

Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Positive Feedback Count       0
Division Name                14
Department Name              14
Category                     14
Recommended IND               0
dtype: int64

In [4]:
review.shape

(23486, 10)

In [5]:
review.dropna(inplace = True)

In [6]:
review.shape

(19662, 10)

Data with null values for any column has been dropped

In [7]:
tot_review = review['Clothing ID'].value_counts()

In [8]:
review['tot_review'] = 0

for unq in review['Clothing ID'].unique():
    review.loc[review['Clothing ID'] == unq, 'tot_review'] = review['Clothing ID'].value_counts()[unq]

In [9]:
review.head()

,Clothing ID,Age,Title,Review Text,Rating,Positive Feedback Count,Division Name,Department Name,Category,Recommended IND,tot_review
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses,0,251
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,0,General Petite,Bottoms,Pants,1,25
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,6,General,Tops,Blouses,1,4
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,4,General,Dresses,Dresses,0,241
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,General Petite,Tops,Knits,1,18


In [10]:
review = review[review['tot_review'] > 5]

In [11]:
review.shape

(18206, 11)

In [12]:
contractions={"a'ight":"alright",
"ain't":"are not",
"amn't":"am not",
"aren't":"are not",
"can't":"cannot",
"'cause": "because",
"could've":"could have",
"couldn't":"could not",
"couldn't've":"could not have",
"daren't":"dare not",
"daresn't":"dare not",
"dasn't":"dare not",
"didn't":"did not",
"doesn't":"does not",
"don't":"do not",
"everybody's":"everybody is",
"everyone's":"everyone is",
"giv'n":"given",
"gonna":"going to",
"gon't":"go not", 
"gotta":"got to",
"hadn't":"had not",
"had've":"had have",
"hasn't":"has not",
"haven't":"have not",
"he'd":	"he had", 
"he'll":"he will",
"he's":	"he is",
"here's":"here is",
"how'd":"how did",
"how'll":"how will",
"how're":"how are",
"how's":"how is",
"I'd":"I had",
"I'd've":"I would have",
"I'd'nt":"I would not",
"I'd'nt've":"I would not have",
"I'll":"I will",
"I'm":"I am",
"I've":"I have",
"isn't":"is not",
"it'd":"it would",
"it'll":"it will",
"it's":	"it is",
"let's":"let us",
"ma'am":"madam",
"mayn't":"may not",
"may've":"may have",
"mightn't":"might not",
"might've":"might have",
"mustn't":"must not",
"mustn't've":"must not have",
"must've":"must have",
"needn't":"need not",
"needn't've":"need not have",
"o'clock":"of the clock",
"oughtn't":"ought not",
"oughtn't've":"ought not have",
"shan't":"shall not",
"she'd":"she would",
"she'll":"she will",
"she's":"she is",
"should've":"should have",
"shouldn't":"should not",
"shouldn't've":"should not have",
"somebody's":"somebody is",
"someone's":"someone is",
"something's":"something is",
"so're":"so are",
"so’s":"so is",
"so’ve":"so have",
"that'll":"that will",
"that're":"that are",
"that's":"that is",
"that'd":"that would",
"there'd":"there would",
"there'll":"there will",
"there're":"there are",
"there's":"there is",
"these're":"these are",
"these've":"these have",
"they'd":"they would",
"they'll":"they will",
"they're":"they are",
"they've":"they have",
"this's":"this is",
"those're":"those are",
"those've":"those have",
"to've":"to have",
"wasn't":"was not",
"we'd":	"we would",
"we'd've":"we would have",
"we'll":"we will",
"we're":"we are",
"we've":"we have",
"weren't":"were not",
"what'd":"what did",
"what'll":"what will",
"what're":"what are",
"what's":"what is",
"what've":"what have",
"when's":"when is",
"where'd":"where did",
"where'll":"where will",
"where're":"where are",
"where's":"where is",
"where've":"where have",
"which'd":"which would",
"which'll":"which will",
"which're":"which are",
"which's":"which is",
"which've":"which have",
"who'd":"who would",
"who'd've":"who would have",
"who'll":"who will",
"who're":"who are",
"who's":"who is",
"who've":"who have",
"why'd":"why did",
"why're":"why are",
"why's":"why is",
"won't":"will not",
"would've":"would have",
"wouldn't":"would not",
"wouldn't've":"would not have",
"y'at":	"you at",
"yes’m":"yes madam",
"you'd":"you would",
"you'll":"you will",
"you're":"you are",
"you've":"you have"}

In [13]:
review['Review Text']

2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
5        I love tracy reese dresses, but this one is no...
6        I aded this in my basket at hte last mintue to...
7        I ordered this in carbon for store pick up, an...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 18206, dtype: object

In [14]:
import string

In [15]:
def rem_punc(x):
    punc = []
    punc[:0] = string.punctuation
    for p in punc:
        x = x.replace(p, '')
    return x

In [16]:
def cont_to_exp(x):
    for key in contractions:
        value = contractions[key]
        x = x.replace(key, value)
    return x

In [17]:
review['Review Text'] = review['Review Text'].str.lower()
review['Review Text'] = review['Review Text'].apply(lambda x : rem_punc(x))
review['Review Text'] = review['Review Text'].apply(lambda x : cont_to_exp(x))

In [18]:
review['Review Text']

2        i had such high hopes for this dress and reall...
3        i love love love this jumpsuit its fun flirty ...
5        i love tracy reese dresses but this one is not...
6        i aded this in my basket at hte last mintue to...
7        i ordered this in carbon for store pick up and...
                               ...                        
23481    i was very happy to snag this dress at such a ...
23482    it reminds me of maternity clothes soft stretc...
23483    this fit well but the top was very see through...
23484    i bought this dress for a wedding i have this ...
23485    this dress in a lovely platinum is feminine an...
Name: Review Text, Length: 18206, dtype: object

## 2. Sentiment Catagory

In [19]:
from textblob import TextBlob

In [20]:
review['polarity'] = review['Review Text'].apply(lambda x : TextBlob(x).sentiment.polarity)

In [21]:
review.head()

,Clothing ID,Age,Title,Review Text,Rating,Positive Feedback Count,Division Name,Department Name,Category,Recommended IND,tot_review,polarity
2,1077,60,Some major design flaws,i had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses,0,251,0.076392
3,1049,50,My favorite buy!,i love love love this jumpsuit its fun flirty ...,5,0,General Petite,Bottoms,Pants,1,25,0.500000
5,1080,49,Not for the very petite,i love tracy reese dresses but this one is not...,2,4,General,Dresses,Dresses,0,241,0.178750
6,858,39,Cagrcoal shimmer fun,i aded this in my basket at hte last mintue to...,5,1,General Petite,Tops,Knits,1,18,0.060500
7,858,39,"Shimmer, surprisingly goes with lots",i ordered this in carbon for store pick up and...,4,4,General Petite,Tops,Knits,1,18,0.171635


In [22]:
review['Sentiment Category'] = 0

In [23]:
for i in range(review.index.shape[0]):
    if review.loc[review.index[i], 'polarity'] >= 0.5:
        review.loc[review.index[i], 'Sentiment Category'] = 4
    elif review.loc[review.index[i], 'polarity'] >= 0:
        review.loc[review.index[i], 'Sentiment Category'] = 3
    elif review.loc[review.index[i], 'polarity'] <= -0.5:
        review.loc[review.index[i], 'Sentiment Category'] = 2
    elif review.loc[review.index[i], 'polarity']  < 0:
        review.loc[review.index[i], 'Sentiment Category'] = 1

In [24]:
review.head()

,Clothing ID,Age,Title,Review Text,Rating,Positive Feedback Count,Division Name,Department Name,Category,Recommended IND,tot_review,polarity,Sentiment Category
2,1077,60,Some major design flaws,i had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses,0,251,0.076392,3
3,1049,50,My favorite buy!,i love love love this jumpsuit its fun flirty ...,5,0,General Petite,Bottoms,Pants,1,25,0.500000,4
5,1080,49,Not for the very petite,i love tracy reese dresses but this one is not...,2,4,General,Dresses,Dresses,0,241,0.178750,3
6,858,39,Cagrcoal shimmer fun,i aded this in my basket at hte last mintue to...,5,1,General Petite,Tops,Knits,1,18,0.060500,3
7,858,39,"Shimmer, surprisingly goes with lots",i ordered this in carbon for store pick up and...,4,4,General Petite,Tops,Knits,1,18,0.171635,3


In [25]:
review['Sentiment Category'].unique()

array([3, 4, 1, 2], dtype=int64)

## 3. Naive Bayes Classification

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [27]:
review.columns

Index(['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Positive Feedback Count', 'Division Name', 'Department Name',
       'Category', 'Recommended IND', 'tot_review', 'polarity',
       'Sentiment Category'],
      dtype='object')

In [28]:
x = np.array(review['Sentiment Category']).reshape(-1,1)
y = review['Recommended IND']

In [29]:
x

array([[3],
       [4],
       [3],
       ...,
       [3],
       [3],
       [3]], dtype=int64)

In [30]:
y.head()

2    0
3    1
5    0
6    1
7    1
Name: Recommended IND, dtype: int64

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 105) 
model = MultinomialNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

## 4. Accuracy Measures

In [32]:
from sklearn import metrics

In [33]:
metrics.accuracy_score(y_test, y_pred)

0.8220757825370676

In [34]:
metrics.precision_score(y_test, y_pred)

0.8220757825370676

In [35]:
metrics.recall_score(y_test, y_pred)

1.0

In [36]:
metrics.f1_score(y_test, y_pred)

0.9023508137432189